<a href="https://colab.research.google.com/github/rahaveep/RESUME/blob/main/rprabak1_asu_edu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from xgboost import XGBRegressor
! pip install xgboost
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import absolute
import xgboost

In [ ]:
production_data=pd.read_csv('/content/sample_data/Production Quantity.csv')
precipitation_data=pd.read_csv('/content/sample_data/Daily Precipitation.csv')
soilmoisture_data=pd.read_csv('/content/sample_data/Daily Soil Mositure.csv')
temperature_data=pd.read_csv('/content/sample_data/Daily Temperature.csv')
ndvi_data=pd.read_csv('/content/sample_data/Eight Day NDVI.csv')


Converting start date from daily basis frequency to monthly basis. 
Since the final output depends mostly on the production every month, the start dates of the various csv files are converted into month-Year format. Also taking the mean value of the other variables and assigning it on monthly basis. 

In [ ]:
precipitation_data.index = pd.to_datetime(precipitation_data.start_date)
temperature_data.index = pd.to_datetime(temperature_data.start_date)
soilmoisture_data.index = pd.to_datetime(soilmoisture_data.start_date)
ndvi_data.index = pd.to_datetime(ndvi_data.start_date)
df = precipitation_data.groupby([precipitation_data.index.strftime('%m-%Y'),precipitation_data.region_id],sort=False)['precip'].mean().reset_index()
df1=temperature_data.groupby([temperature_data.index.strftime('%m-%Y'),temperature_data.region_id],sort=False)['temp'].mean().reset_index()
df2=soilmoisture_data.groupby([soilmoisture_data.index.strftime('%m-%Y'),soilmoisture_data.region_id],sort=False)['smos'].mean().reset_index()
df3=ndvi_data.groupby([ndvi_data.index.strftime('%m-%Y'),ndvi_data.region_id],sort=False)['ndvi'].mean().reset_index()

In [ ]:
df=df.loc[df.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020','2021'])]
df1=df1.loc[df1.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020','2021'])]
df2=df2.loc[df2.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020','2021'])]
df3=df3.loc[df3.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020','2021'])]


In [ ]:
df=df.reset_index(drop=True)
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)
df3=df3.reset_index(drop=True)

In [ ]:
df1.drop('region_id',axis=1,inplace=True)
df2.drop('region_id',axis=1,inplace=True)
df3.drop('region_id',axis=1,inplace=True)


In [ ]:
dataset = pd.concat([df,df1['temp'],df2['smos'],df3['ndvi']],axis=1,sort=False)

In [ ]:
production_data.index = pd.to_datetime(production_data.start_date)
production_data.index=production_data.index.strftime('%m-%Y')
production_data.rename(columns = {'index':'start_date'},inplace=True)
production_data['start_date']=production_data.index
dataset['prod'] = dataset.start_date.map(production_data.set_index('start_date')['prod'].to_dict())


In [ ]:
dataset.fillna(value=dataset['ndvi'].mean(), inplace=True)

Obtaining the master dataset whose start_date is on monthly basis and Seperating the dataset on basis of different region ids.

In [ ]:
dataset

,start_date,region_id,precip,temp,smos,ndvi,prod
0,01-2015,93,4.394452,25.003593,0.318552,0.761228,19382.000000
1,02-2015,93,1.206137,26.807568,0.291452,0.758957,17106.000000
2,03-2015,93,3.454652,26.954109,0.251969,0.752754,25482.000000
3,04-2015,93,10.466472,26.948865,0.248915,0.785057,36875.000000
4,05-2015,93,9.007986,25.405121,0.280602,0.805262,45605.000000
...,...,...,...,...,...,...,...
835,08-2021,105,7.829626,26.764261,0.267808,0.801862,0.800874
836,09-2021,105,8.859617,26.673888,0.251307,0.800874,0.800874
837,10-2021,105,7.305735,26.967615,0.277053,0.800874,0.800874
838,11-2021,105,16.057218,25.322988,0.397742,0.800874,0.800874


In [ ]:
d1 = dataset[dataset['region_id'] == 93]
d2=dataset[dataset['region_id']==94]
d3=dataset[dataset['region_id']==95]
d4=dataset[dataset['region_id']==97]
d5=dataset[dataset['region_id']==98]
d6=dataset[dataset['region_id']==99]
d7=dataset[dataset['region_id']==102]
d8=dataset[dataset['region_id']==103]
d9=dataset[dataset['region_id']==104]
d10=dataset[dataset['region_id']==105]





Obtaining the train and test dataset

In [ ]:
train=d1.loc[d1.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020'])]
test=d1.loc[d1.start_date.str.split('-',expand=True)[1].isin(['2021'])]

Training the model using XGBoost and calculating the mean absolute error and R2 Score for each region id dataset.

In [ ]:
x_train=train.drop(['start_date','prod'],axis=1)
y_train=train['prod']
x_test=test.drop(['start_date','prod'],axis=1)
true_p=test['prod']
Ireg = XGBRegressor()
Ireg= XGBRegressor(n_estimators=100, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)
Ireg.fit(x_train,y_train)
d1_predict=Ireg.predict(x_test)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(Ireg, x_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = absolute(scores)
#print('Mean MAE: %.3f (%.3f)' % (scores1.mean(), scores1.std()) )
#print(sklearn.metrics.r2_score(true_p, d1_predict))
Ireg.score(x_test,true_p)
Ireg.score(x_train,y_train)

[23:04:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.996818672302953

In [ ]:
train2=d2.loc[d2.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020'])]
test2=d2.loc[d2.start_date.str.split('-',expand=True)[1].isin(['2021'])]
x_train2=train2.drop(['start_date','prod'],axis=1)
y_train2=train2['prod']
x_test2=test2.drop(['start_date','prod'],axis=1)
true_p2=test2['prod']
Ireg = XGBRegressor()
Ireg= XGBRegressor(n_estimators=100, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)
Ireg.fit(x_train,y_train)
d2_predict=Ireg.predict(x_test)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores2 = cross_val_score(Ireg, x_train2, y_train2, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores2 = absolute(scores2)
#print('Mean MAE: %.3f (%.3f)' % (scores2.mean(), scores2.std()) )
#print(sklearn.metrics.r2_score(true_p2, d2_predict))
Ireg.score(x_test2,true_p2)
Ireg.score(x_train2,y_train2)

[23:04:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


-0.3931717634739924

In [ ]:
train3=d3.loc[d3.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020'])]
test3=d3.loc[d3.start_date.str.split('-',expand=True)[1].isin(['2021'])]
x_train3=train3.drop(['start_date','prod'],axis=1)
y_train3=train3['prod']
x_test3=test3.drop(['start_date','prod'],axis=1)
true_p3=test3['prod']
Ireg = XGBRegressor()
Ireg= XGBRegressor(n_estimators=100, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)
Ireg.fit(x_train3,y_train3)
d3_predict=Ireg.predict(x_test3)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores3 = cross_val_score(Ireg, x_train3, y_train3, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores3 = absolute(scores3)
#print('Mean MAE: %.3f (%.3f)' % (scores3.mean(), scores3.std()) )
#print(sklearn.metrics.r2_score(true_p3, d3_predict))
Ireg.fit(x_train3,y_train3)
d3_predict=Ireg.predict(x_test3)
Ireg.score(x_test3,true_p3)
Ireg.score(x_train3,y_train3)

[23:04:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:04:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9983862124275773

In [ ]:
train4=d4.loc[d4.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020'])]
test4=d4.loc[d4.start_date.str.split('-',expand=True)[1].isin(['2021'])]
x_train4=train4.drop(['start_date','prod'],axis=1)
y_train4=train4['prod']
x_test4=test4.drop(['start_date','prod'],axis=1)
true_p4=test4['prod']
Ireg = XGBRegressor()
Ireg= XGBRegressor(n_estimators=100, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)
Ireg.fit(x_train4,y_train4)
d4_predict=Ireg.predict(x_test4)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores4 = cross_val_score(Ireg, x_train4, y_train4, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores4 = absolute(scores4)
#print('Mean MAE: %.3f (%.3f)' % (scores4.mean(), scores4.std()) )
#print(sklearn.metrics.r2_score(true_p4, d4_predict))
Ireg.fit(x_train4,y_train4)
d4_predict=Ireg.predict(x_test4)
Ireg.score(x_test4,true_p4)
Ireg.score(x_train4,y_train4)

[23:04:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:04:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.990697508789039

In [ ]:
train5=d5.loc[d5.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020'])]
test5=d5.loc[d5.start_date.str.split('-',expand=True)[1].isin(['2021'])]
x_train5=train5.drop(['start_date','prod'],axis=1)
y_train5=train5['prod']
x_test5=test5.drop(['start_date','prod'],axis=1)
true_p5=test5['prod']
Ireg = XGBRegressor()
Ireg= XGBRegressor(n_estimators=100, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)
Ireg.fit(x_train5,y_train5)
d5_predict=Ireg.predict(x_test5)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores5 = cross_val_score(Ireg, x_train5, y_train5, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores5 = absolute(scores5)
#print('Mean MAE: %.3f (%.3f)' % (scores5.mean(), scores5.std()) )
#print(sklearn.metrics.r2_score(true_p5, d5_predict))
Ireg.fit(x_train5,y_train5)
d5_predict=Ireg.predict(x_test5)
Ireg.score(x_test5,true_p5)
Ireg.score(x_train5,y_train5)

[23:04:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:04:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9982674158319049

In [ ]:
train6=d6.loc[d6.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020'])]
test6=d6.loc[d6.start_date.str.split('-',expand=True)[1].isin(['2021'])]
x_train6=train6.drop(['start_date','prod'],axis=1)
y_train6=train6['prod']
x_test6=test6.drop(['start_date','prod'],axis=1)
true_p6=test6['prod']
Ireg = XGBRegressor()
Ireg= XGBRegressor(n_estimators=100, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)
Ireg.fit(x_train6,y_train6)
d6_predict=Ireg.predict(x_test6)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores6 = cross_val_score(Ireg, x_train6, y_train6, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores6 = absolute(scores6)
#print('Mean MAE: %.3f (%.3f)' % (scores6.mean(), scores6.std()) )
#print(sklearn.metrics.r2_score(true_p6, d6_predict))
Ireg.fit(x_train6,y_train6)
d6_predict=Ireg.predict(x_test6)
Ireg.score(x_test6,true_p6)
Ireg.score(x_train6,y_train6)

[23:04:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:04:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9946471745589104

In [ ]:
train7=d7.loc[d7.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020'])]
test7=d7.loc[d7.start_date.str.split('-',expand=True)[1].isin(['2021'])]
x_train7=train7.drop(['start_date','prod'],axis=1)
y_train7=train7['prod']
x_test7=test7.drop(['start_date','prod'],axis=1)
true_p7=test7['prod']
Ireg = XGBRegressor()
Ireg= XGBRegressor(n_estimators=100, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)
Ireg.fit(x_train7,y_train7)
d7_predict=Ireg.predict(x_test7)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores7 = cross_val_score(Ireg, x_train7, y_train7, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores7 = absolute(scores7)
#print('Mean MAE: %.3f (%.3f)' % (scores7.mean(), scores7.std()) )
#print(sklearn.metrics.r2_score(true_p7, d7_predict))
Ireg.fit(x_train7,y_train7)
d7_predict=Ireg.predict(x_test7)
Ireg.score(x_test7,true_p7)
Ireg.score(x_train7,y_train7)

[23:04:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:04:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9937506377849745

In [ ]:
train8=d8.loc[d8.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020'])]
test8=d8.loc[d8.start_date.str.split('-',expand=True)[1].isin(['2021'])]
x_train8=train8.drop(['start_date','prod'],axis=1)
y_train8=train8['prod']
x_test8=test8.drop(['start_date','prod'],axis=1)
true_p8=test8['prod']
Ireg = XGBRegressor()
Ireg= XGBRegressor(n_estimators=100, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)
Ireg.fit(x_train8,y_train8)
d8_predict=Ireg.predict(x_test8)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores8 = cross_val_score(Ireg, x_train8, y_train8, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores8 = absolute(scores8)
#print('Mean MAE: %.3f (%.3f)' % (scores8.mean(), scores8.std()) )
#print(sklearn.metrics.r2_score(true_p8, d8_predict))
Ireg.fit(x_train8,y_train8)
d8_predict=Ireg.predict(x_test8)
Ireg.score(x_test8,true_p8)
Ireg.score(x_train8,y_train8)

[23:04:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:04:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9968526410077039

In [ ]:
train9=d9.loc[d9.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020'])]
test9=d9.loc[d9.start_date.str.split('-',expand=True)[1].isin(['2021'])]
x_train9=train9.drop(['start_date','prod'],axis=1)
y_train9=train9['prod']
x_test9=test9.drop(['start_date','prod'],axis=1)
true_p9=test9['prod']
Ireg = XGBRegressor()
Ireg= XGBRegressor(n_estimators=100, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)
Ireg.fit(x_train9,y_train9)
d9_predict=Ireg.predict(x_test9)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores9 = cross_val_score(Ireg, x_train9, y_train9, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores9 = absolute(scores9)
#print('Mean MAE: %.3f (%.3f)' % (scores9.mean(), scores9.std()) )
#print(sklearn.metrics.r2_score(true_p9, d1_predict9))
Ireg.fit(x_train9,y_train9)
d9_predict=Ireg.predict(x_test9)
Ireg.score(x_test9,true_p9)
Ireg.score(x_train9,y_train9)

[23:04:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:04:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.994981305468572

In [ ]:
train10=d10.loc[d10.start_date.str.split('-',expand=True)[1].isin(['2015','2016','2017','2018','2019','2020'])]
test10=d10.loc[d10.start_date.str.split('-',expand=True)[1].isin(['2021'])]
x_train10=train10.drop(['start_date','prod'],axis=1)
y_train10=train10['prod']
x_test10=test10.drop(['start_date','prod'],axis=1)
true_p10=test10['prod']
Ireg = XGBRegressor()
Ireg= XGBRegressor(n_estimators=100, max_depth=5, eta=0.1, subsample=0.7, colsample_bytree=0.8)
Ireg.fit(x_train10,y_train10)
d10_predict=Ireg.predict(x_test10)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores10 = cross_val_score(Ireg, x_train10, y_train10, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores10 = absolute(scores10)
#print('Mean MAE: %.3f (%.3f)' % (scores10.mean(), scores10.std()) )
#print(sklearn.metrics.r2_score(true_p10, d10_predict))
Ireg.fit(x_train10,y_train10)
d10_predict=Ireg.predict(x_test10)
Ireg.score(x_test10,true_p10)
Ireg.score(x_train10,y_train10)

[23:04:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:04:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9976531782519825

In [ ]:
x_test_final=test
x_test_final.drop(['start_date','prod'],axis=1,inplace=True)
prediction_final=Ireg.predict(x_test_final)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
x_test_final2=test2
test2.drop(['start_date','prod'],axis=1,inplace=True)
prediction_final2=Ireg.predict(x_test_final2)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
x_test_final3=test3
x_test_final3.drop(['start_date','prod'],axis=1,inplace=True)
prediction_final3=Ireg.predict(x_test_final3)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
x_test_final4=test4
x_test_final4.drop(['start_date','prod'],axis=1,inplace=True)
prediction_final4=Ireg.predict(x_test_final4)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
x_test_final5=test5
x_test_final5.drop(['start_date','prod'],axis=1,inplace=True)
prediction_final5=Ireg.predict(x_test_final5)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
x_test_final6=test6
x_test_final6.drop(['start_date','prod'],axis=1,inplace=True)
prediction_final6=Ireg.predict(x_test_final6)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
x_test_final7=test7
x_test_final7.drop(['start_date','prod'],axis=1,inplace=True)
prediction_final7=Ireg.predict(x_test_final7)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
x_test_final8=test8
x_test_final8.drop(['start_date','prod'],axis=1,inplace=True)
prediction_final8=Ireg.predict(x_test_final8)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
x_test_final9=test9
x_test_final9.drop(['start_date','prod'],axis=1,inplace=True)
prediction_final9=Ireg.predict(x_test_final9)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
x_test_final10=test10
x_test_final10.drop(['start_date','prod'],axis=1,inplace=True)
prediction_final10=Ireg.predict(x_test_final10)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
pred_df1=pd.DataFrame(prediction_final)
pred_df2=pd.DataFrame(prediction_final2)
pred_df3=pd.DataFrame(prediction_final3)
pred_df4=pd.DataFrame(prediction_final4)
pred_df5=pd.DataFrame(prediction_final5)
pred_df6=pd.DataFrame(prediction_final6)
pred_df7=pd.DataFrame(prediction_final7)
pred_df8=pd.DataFrame(prediction_final8)
pred_df9=pd.DataFrame(prediction_final9)
pred_df10=pd.DataFrame(prediction_final10)


In [ ]:
frames = [pred_df1, pred_df2, pred_df3,pred_df4,pred_df5,pred_df6,pred_df7,pred_df8,pred_df9,pred_df10]
result = pd.concat(frames)

In [ ]:
results=pd.DataFrame(result)

In [ ]:
sa=pd.read_csv('/content/sample_data/predicted_production_qty.csv')
sa.drop('prod',axis=1,inplace=True)

In [ ]:
final_dataframe=sa.join(result, lsuffix="_left", rsuffix="_right")

In [ ]:
final_dataframe.fillna(value=dataset['prod'].mean(), inplace=True)

In [ ]:
columns=df.columns.tolist()

In [ ]:
new_cols=['start_date','end_date','region_id','prod']

In [ ]:
final_dataframe.columns = new_cols

In [ ]:
final_dataframe = final_dataframe[['start_date','end_date','prod','region_id']]

In [ ]:
final_dataframe.reset_index(inplace=True)

In [ ]:
final_dataframe.drop('index',axis=1,inplace=True)

In [ ]:
final_dataframe = final_dataframe.apply(lambda x: x.astype(str).str.lower()).drop_duplicates(subset=['start_date', 'region_id'], keep='first')

In [ ]:
final_dataframe

,start_date,end_date,prod,region_id
0,2021-01-01t00:00:00.000z,2021-01-31t00:00:00.000z,41250.676,93
10,2021-02-01t00:00:00.000z,2021-02-28t00:00:00.000z,32383.746,93
20,2021-03-01t00:00:00.000z,2021-03-31t00:00:00.000z,55942.273,93
30,2021-04-01t00:00:00.000z,2021-04-30t00:00:00.000z,48840.625,93
40,2021-05-01t00:00:00.000z,2021-05-31t00:00:00.000z,49447.395,93
...,...,...,...,...
223,2021-08-01t00:00:00.000z,2021-08-31t00:00:00.000z,35967.28,105
224,2021-09-01t00:00:00.000z,2021-09-30t00:00:00.000z,35967.28,105
225,2021-10-01t00:00:00.000z,2021-10-31t00:00:00.000z,35967.28,105
226,2021-11-01t00:00:00.000z,2021-11-30t00:00:00.000z,35967.28,105


In [ ]:
final_dataframe.to_csv('rprabak1@asu.edu.csv',index=False)

To make the model more efficient ,
1. The region_id can be converted into categorical variables.
2. The distribution of the data should be better analysed so another better model can be chosen to get better results. 
